In [42]:
# Dependence

from uuid import uuid1
import uuid
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
from gtts import gTTS  
from pygame import mixer

In [43]:
# initialize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

In [44]:
# Load the gesture recognizer model
model = load_model('mp_hand_gesture')

In [45]:
# Load class names
f = open('gesture.names', 'r')
classNames = f.read().split('\n')
f.close()
print(classNames)
# ['okay', 'thank you', 'Yes', 'No', 'call me', 'stop', 'Love you', 'live long', 'fist', 'end']

['okay', 'thank you', 'Yes', 'No', 'call me', 'stop', 'Love you', 'live long', 'fist', 'end']


In [46]:
# Initialize the webcam
cap = cv2.VideoCapture(0)

In [47]:
# hand gesture recognition to speech.
inProgress= True
while inProgress:
    # Read each nframe from the webcam
    _, frame = cap.read()

    x, y, c = frame.shape

    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get hand landmark prediction
    result = hands.process(framergb)

    # print(result)
    
    className = ''


    # post process the result
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                # print(id, lm)
                lmx = int(lm.x * x)
                lmy = int(lm.y * y)

                landmarks.append([lmx, lmy])

            # Drawing landmarks on frames
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

            # Predict gesture
            prediction = model.predict([landmarks])
            # print(prediction)
            classID = np.argmax(prediction)
            className = classNames[classID]
            temp =''
            
            if className == 'end':
                inProgress=False

            # language = 'en'  
            # if className != temp: 
            #     id = uuid.uuid4()
            #     temp = className
            #     print(temp)
            #     obj = gTTS(text=temp, lang=language, slow=False)  
    
            #     #Here we are saving the transformed audio in a mp3 file named  
            #     # exam.mp3  
            #     try:
            #         obj.save(f'{id}.mp3')  
            #     except Exception:
            #         print(Exception)

            #     mixer.init()
            #     mixer.music.load(f'D:/Downloads/hand-gesture-recognition-code/{id}.mp3')
            #     mixer.music.play()
            
    # show the prediction on the frame
    cv2.putText(frame, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (0,0,255), 2, cv2.LINE_AA)

    # Show the final output
    cv2.imshow("Output", frame) 

    if cv2.waitKey(1) == ord('q'):
        break

In [ ]:
# release the webcam and destroy all active windows
cap.release()

cv2.destroyAllWindows()